In [1]:
import time

# get the start time
st = time.time()

In [2]:
# main program
# BH Simulation
import strawberryfields as sf
import numpy as np
import math

eng = sf.RemoteEngine("borealis")
device = eng.device

In [3]:
device.certificate

{'finished_at': '2022-08-25T17:42:20.550132+00:00',
 'target': 'borealis',
 'loop_phases': [-1.705, 1.814, 0.382],
 'schmidt_number': 1.157,
 'common_efficiency': 0.434,
 'loop_efficiencies': [0.892, 0.893, 0.833],
 'squeezing_parameters_mean': {'low': 0.637, 'high': 1.114, 'medium': 0.935},
 'relative_channel_efficiencies': [0.953,
  0.992,
  0.965,
  0.957,
  0.951,
  0.891,
  0.912,
  0.982,
  0.975,
  1.0,
  0.906,
  0.938,
  0.928,
  0.91,
  0.96,
  0.905]}

In [4]:
from strawberryfields.tdm import borealis_gbs, full_compile, get_mode_indices

gate_args_list = borealis_gbs(device, modes=216, squeezing="high")

In [5]:
modes = 216

# squeezing-gate parameters
r = [1.234] * modes

In [6]:
import numpy as np

# set the random seed
np.random.seed(42)

import strawberryfields as sf
from strawberryfields.ops import *

ham_simulation = sf.Program(2)

# set the Hamiltonian parameters
J = 2        # hopping transition
U = 4       # on-site interaction
k = 100        # Lie product decomposition terms
t = 0.5     # timestep
theta = -J*t/k
r = -U*t/(2*k)

with ham_simulation.context as q:
    # prepare the initial state
    Fock(2) | q[0]

    # Two node tight-binding
    # Hamiltonian simulation

    for i in range(k):
        BSgate(theta, np.pi/4) | (q[0], q[1])
        Kgate(r)  | q[0]
        Rgate(-r) | q[1]
        Kgate(r)  | q[0]
        Rgate(-r) | q[1]

In [7]:
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 5})


In [8]:
results = eng.run(ham_simulation)


In [9]:
# wait for 5 seconds
time.sleep(5)

In [10]:
state = results.state
print("P(|0, 2>) = ", state.fock_prob([0, 2]))
print("P(|1, 1>) = ", state.fock_prob([1, 1]))
print("P(|2, 0>) = ", state.fock_prob([2, 0]))

P(|0, 2>) =  0.006123830455131159
P(|1, 1>) =  0.05971765105003921
P(|2, 0>) =  0.9341585184948159


In [11]:
result = [state.fock_prob([0, 2]), state.fock_prob([1, 1]), state.fock_prob([2, 0])]
print(np.sum(result))

0.9999999999999862


In [12]:
from scipy.linalg import expm

H = J*np.sqrt(2)*np.array([[0,1,0],[1,0,1],[0,1,0]]) + U*np.diag([1,0,1])
init_state = np.array([0,0,1])

theoretical_result = np.abs(np.dot(expm(-1j*t*H), init_state))**2
print(theoretical_result)


[0.3949285  0.24758968 0.35748182]


In [13]:
print(np.all(np.abs(theoretical_result - result) < 1e-2))

False


In [14]:
# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 10.019969701766968 seconds


In [15]:
fidelity = math.sqrt(results.state.fidelity_coherent([0.7+1.2j, 0.7+1.2j]))
print(fidelity)

0.2053677945343228
